In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/multiclass-weather-dataset/dataset/test.csv
/kaggle/input/multiclass-weather-dataset/dataset/cloudy/cloudy88.jpg
/kaggle/input/multiclass-weather-dataset/dataset/cloudy/cloudy167.jpg
/kaggle/input/multiclass-weather-dataset/dataset/cloudy/cloudy78.jpg
/kaggle/input/multiclass-weather-dataset/dataset/cloudy/cloudy80.jpg
/kaggle/input/multiclass-weather-dataset/dataset/cloudy/cloudy2.jpg
/kaggle/input/multiclass-weather-dataset/dataset/cloudy/cloudy89.jpg
/kaggle/input/multiclass-weather-dataset/dataset/cloudy/cloudy99.jpg
/kaggle/input/multiclass-weather-dataset/dataset/cloudy/cloudy256.jpg
/kaggle/input/multiclass-weather-dataset/dataset/cloudy/cloudy9.jpg
/kaggle/input/multiclass-weather-dataset/dataset/cloudy/cloudy185.jpg
/kaggle/input/multiclass-weather-dataset/dataset/cloudy/cloudy208.jpg
/kaggle/input/multiclass-weather-dataset/dataset/cloudy/cloudy44.jpg
/kaggle/input/multiclass-weather-dataset/dataset/cloudy/cloudy269.jpg
/kaggle/input/multiclass-weather-dataset/d

In [2]:
data_dir="/kaggle/input/multiclass-weather-dataset/dataset"

In [3]:
#eassy interface for UI Gradio provides tools for visualizing input data and model predictions, making it easier to understand how your model is performing

!pip install gradio

  Preparing metadata (setup.py) ... - done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.9/16.9 MB 81.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 307.7/307.7 kB 14.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 97.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.9/75.9 kB 3.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.0/77.0 kB 4.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 129.9/129.9 kB 7.9 MB/s eta 0:00:00
  Created wheel for ffmpy: filename=ffmpy-0.3.2-py3-none-any.whl size=5584 sha256=8e30f2ba4d60ffd3f13232132f03c4dafd27ad332057bc5053f388ab10fa5008
  Stored in directory: /root/.cache/pip/wheels/bd/65/9a/671fc6dcde07d4418df0c592f8df512b26d7a0029c2a23dd81
Successfully built ffmpy
  Attempting uninstall: websockets
    Found existing installation: websockets 12.0
    Uninstalling websockets-12.0:
      Successfully uninstalled websockets-12.0
  Attempting uninstall: tomlkit
    Fo

In [4]:
import gradio as gr
import matplotlib.pyplot as plt
import numpy as np
import os
import PIL
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

2024-02-17 08:14:03.555270: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-02-17 08:14:03.555387: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-02-17 08:14:03.702753: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [5]:
img_height,img_width=180,180
batch_size=32
train_ds = tf.keras.preprocessing.image_dataset_from_directory(
    data_dir,
    validation_split=0.2,
    subset="training",
    seed=123,
    image_size=(img_height, img_width),
    batch_size=batch_size
)

Found 1530 files belonging to 6 classes.
Using 1224 files for training.


In [6]:
val_ds = tf.keras.preprocessing.image_dataset_from_directory(
    data_dir,
    validation_split=0.2,
    subset="validation",
    seed=123,
    image_size=(img_height, img_width),
    batch_size=batch_size
)

Found 1530 files belonging to 6 classes.
Using 306 files for validation.


In [7]:
class_names = train_ds.class_names
print(class_names)

['alien_test', 'cloudy', 'foggy', 'rainy', 'shine', 'sunrise']


In [8]:
num_classes = len(class_names)

In [9]:
from tensorflow.keras.models import Sequential
from tensorflow.keras import layers

model = Sequential([
    layers.experimental.preprocessing.Rescaling(1./255, input_shape=(img_height, img_width, 3)),
    layers.Conv2D(16, 3, padding='same', activation='relu'),
    layers.MaxPooling2D(),
    layers.Conv2D(32, 3, padding='same', activation='relu'),
    layers.MaxPooling2D(),
    layers.Conv2D(64, 3, padding='same', activation='relu'),
    layers.MaxPooling2D(),
    layers.Flatten(),
    layers.Dense(128, activation='relu'),
    layers.Dense(num_classes, activation='softmax')
])


In [10]:
model.compile(
    optimizer='adam',
    loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
    metrics=['accuracy']
)

In [11]:
epochs = 10
history = model.fit(
    train_ds,
    validation_data=val_ds,
    epochs=epochs
)


Epoch 1/10


/opt/conda/lib/python3.10/site-packages/keras/src/backend.py:5727: UserWarning: "`sparse_categorical_crossentropy` received `from_logits=True`, but the `output` argument was produced by a Softmax activation and thus does not represent logits. Was this intended?
  output, from_logits = _get_logits(


39/39 [==============================] - 19s 421ms/step - loss: 1.3520 - accuracy: 0.5057 - val_loss: 0.9555 - val_accuracy: 0.6601
Epoch 2/10
39/39 [==============================] - 16s 393ms/step - loss: 0.8322 - accuracy: 0.7190 - val_loss: 0.8082 - val_accuracy: 0.7353
Epoch 3/10
39/39 [==============================] - 15s 380ms/step - loss: 0.6726 - accuracy: 0.7688 - val_loss: 0.7793 - val_accuracy: 0.7614
Epoch 4/10
39/39 [==============================] - 16s 393ms/step - loss: 0.5568 - accuracy: 0.8047 - val_loss: 0.8001 - val_accuracy: 0.7516
Epoch 5/10
39/39 [==============================] - 16s 393ms/step - loss: 0.4523 - accuracy: 0.8529 - val_loss: 0.6271 - val_accuracy: 0.8235
Epoch 6/10
39/39 [==============================] - 16s 400ms/step - loss: 0.3575 - accuracy: 0.8881 - val_loss: 0.6630 - val_accuracy: 0.8301
Epoch 7/10
39/39 [==============================] - 16s 394ms/step - loss: 0.2940 - accuracy: 0.9052 - val_loss: 0.7678 - val_accuracy: 0.8007
Epoch 8/10

In [12]:
model.layers
model.metrics_names

['loss', 'accuracy']

In [13]:
train_loss, train_accuracy = model.evaluate(train_ds)
val_loss, val_accuracy = model.evaluate(val_ds)

print('Training Loss:', train_loss)
print('Training Accuracy:', train_accuracy)

print('Validation Loss:', val_loss)
print('Validation Accuracy:', val_accuracy)


10/10 [==============================] - 2s 92ms/step - loss: 1.0689 - accuracy: 0.7255
Training Loss: 0.2175496369600296
Training Accuracy: 0.9166666865348816
Validation Loss: 1.0688730478286743
Validation Accuracy: 0.7254902124404907


In [14]:
from sklearn.metrics import classification_report
import numpy as np

In [15]:
predictions = model.predict(train_ds)
prediction_classes = np.argmax(predictions, axis=1)

39/39 [==============================] - 5s 112ms/step


In [16]:
from sklearn.metrics import classification_report

# Assuming train_ds contains both input data and target labels
# Extract the true labels from train_ds
true_labels = []
for images, labels in train_ds:
    true_labels.extend(labels.numpy())

# Generate classification report using true labels and predicted classes
report = classification_report(true_labels, prediction_classes)
print(report)


              precision    recall  f1-score   support

           0       0.00      0.00      0.00        23
           1       0.21      0.24      0.22       251
           2       0.17      0.16      0.16       223
           3       0.17      0.19      0.18       246
           4       0.14      0.10      0.12       199
           5       0.21      0.22      0.21       282

    accuracy                           0.18      1224
   macro avg       0.15      0.15      0.15      1224
weighted avg       0.18      0.18      0.18      1224

